 ## **Primera practica IACD**
Aris Vazdekis Soria y
Alejandra Ruiz de Adana Fleitas

Librerias

In [1]:
import random
import string

Regularización

In [2]:
def regularizacion(formula, diccionario_comprobador):
    """Comprueba si la formula pasada es correcta o tiene fallas en su escritura"""
    operadores = "!|&>=()"  # Operadores lógicos permitidos
    letras_permitidas = set(string.ascii_lowercase)  # Todas las letras minúsculas
    formula_completa = formula #esto se lo pasaremos a separador para que tome como referencia la formula completa para
                               #cuando generador_aleatorio_letra tome todas las letras y no solo una (este fue el fallo
                               #por el que no nos funcionaban formulas complejas)

    parentesis = []  # Utilizamos una pila para verificar los paréntesis
    for i in range(len(formula)):
        if formula[i] in letras_permitidas or formula[i] in operadores:
            if formula[i] == '!':
                if (formula[i + 1] in letras_permitidas or formula[i + 1] == "!" or formula[i + 1] == "(") and (formula[i - 1] not in letras_permitidas or i == 0 or formula[i - 1] == "!"):
                    continue
                else:
                    print("El operador ! tiene mal asociado su término.")
                    return False

            if formula[i] in ['|', '&', '>', '=']:
                if (formula[i + 1] in letras_permitidas or formula[i + 1] in "(" or  formula[i + 1] in "!") and (formula[i - 1] in letras_permitidas or  formula[i - 1] in ")"):
                    continue
                else:
                    print("El operador " + formula[i] + " tiene mal asociado su término.")
                    return False

            if formula[i] == '(':
                parentesis.append(formula[i])  # Agregar a la pila si encontramos un paréntesis de apertura
            elif formula[i] == ')':
                if '(' not in parentesis:
                    print("La fórmula es incorrecta y no se puede procesar. Paréntesis desbalanceados.")
                    return False
                else:
                    parentesis.pop()  # Si encontramos un paréntesis de cierre, eliminamos uno de la pila
        else:
            print("La fórmula es incorrecta y no se puede procesar. Carácter no permitido:", formula[i])
            return False

    if '(' in parentesis:
        print("La fórmula es incorrecta y no se puede procesar. Paréntesis desbalanceados.")
        return False
    else:
        print("La fórmula es correcta y puede procesarse.")
        formula = separador(formula, diccionario_comprobador, formula_completa)
    return formula

Crear diccionario de vectores


In [3]:
def contador_letras(formula):
    """Esta funcion contara y almacenara en una lista todas las letras que haya
    en la formula"""
    letras_vistas = []  # lista para almacenar las letras que ya hemos visto y evitar duplicados
    for letra in formula:
        if letra.isalpha() and letra not in letras_vistas: #.isalpha() verifica que sea del rango a-z
            letras_vistas.append(letra)
    return letras_vistas

def generador_diccionario(formula):
    """Genera un diccionario conteniendo como claves las letras de la formula y
    como valor su tabla de verdad correspondiente"""
    letras_vistas = contador_letras(formula)
    num_variables = len(letras_vistas)
    print(num_variables)
    num_filas = 2 ** num_variables #puesto que segun el nº de variables la longitud del vector varía
    diccionario_comprobador = {}

    #construccion de las tablas de verdad para cada letra
    for i in range(num_filas):
        fila = []
        for j in range(num_variables):
            fila.append((i // (2 ** j)) % 2) #esta expresion es un truco común para generar secuencias
            #binarias. Básicamente, divide i por 2 elevado a j y toma
            #el residuo de la división por 2. Esto genera una secuencia
            #binaria de valores que se repiten cada vez que i incrementa.
        for k, letra in enumerate(letras_vistas):
            diccionario_comprobador[letra] = diccionario_comprobador.get(letra, []) + [fila[k]]

    return diccionario_comprobador

Buscar las letras en el diccionario

In [4]:
def comprobador( diccionario_comprobador, n, n_1):
    """Comprueba si existe la letra n y devuelve su valor"""
    if n in diccionario_comprobador:
        valor = diccionario_comprobador[n]
    if n_1 in diccionario_comprobador:
        valor = diccionario_comprobador[n_1]
    return valor

Elegir letra aleatoria

In [5]:
def generar_letra_aleatoria(formula):
    """Se utiliza para cada trozo de formula que el programa vaya resolviendo de manera individual,
    asignarle una letra como clave para añadirla al diccionario y poder operar asi con ella"""
    letras_formula = set(c.lower() for c in formula if c.isalpha())
    todas_letras = set(string.ascii_lowercase)
    letras_disponibles = todas_letras - letras_formula
    if letras_disponibles:
        return random.choice(list(letras_disponibles))
    else:
        print("La formula es demasiado grande para procesarse")
        return None  # No hay letras disponibles

Transformar la formula

In [6]:
def borrador(formula, letranueva, letra_actual1, letra_actual2):
    """Utiliza la formula original para modificar cada parte resuelta y actualizarla con su respectiva letra identificadora.
    Ejemplo: (p&!q)|(r&s) ---> a = (p&!q) ---> a|(r&s)"""
    i = 0
    #Iteramos sobre la fórmula para buscar y eliminar la letra actual
    while i < len(formula):
        if formula[i] == letra_actual1 or formula[i] == letra_actual2:
            if formula[i-1] == "!":
                inicio = i - 1
                final = i + 1
                formula = formula[:inicio] + letranueva + formula[final:]
                i = inicio + 1

            if formula[i - 1] == '&' or formula[i - 1] == '|':
                inicio = i - 2
                final = i + 1
                formula = formula[:inicio] + letranueva + formula[final:]
                i = inicio + 1

            if formula[i - 1] == '>' or formula[i - 1] == '=':
                inicio = i - 2
                final = i + 1
                formula = formula[:inicio] + letranueva + formula[final:]
                i = inicio + 1
        i += 1
    return formula

Separar la formula

In [7]:
def separador(formula, diccionario_comprobador, formula_completa):
    """Esta funcion se va a encargar de operar la formula y mandar cada parte de la formula a su
    correspondiente funcion para calcular su resultado"""
    while len(formula) > 1:  # Repetir hasta que solo quede una letra en la fórmula

        while "(" in formula:
            inicio_parentesis = formula.rfind("(")  # Encontrar el último paréntesis de apertura
            fin_parentesis = formula.index(")", inicio_parentesis)  # Encontrar el paréntesis de cierre correspondiente

            # Manejar paréntesis anidados
            contador = 1
            for i in range(inicio_parentesis + 1, len(formula)):
                if formula[i] == "(":
                    contador += 1
                elif formula[i] == ")":
                    contador -= 1
                    if contador == 0:
                        fin_parentesis = i
                        break

            subformula = formula[inicio_parentesis + 1:fin_parentesis]  # Extraer la subfórmula dentro del paréntesis
            resultado_subformula = separador(subformula, diccionario_comprobador, formula_completa)  # Resolver la subfórmula recursivamente
            formula = formula[:inicio_parentesis] + resultado_subformula + formula[fin_parentesis + 1:]  # Reemplazar la subfórmula por su resultado en la fórmula original
            continue  # Continuar con el siguiente ciclo sin procesar el resto de la fórmula actual

        # Manejar las negaciones primero
        while "!" in formula:
            vector1 = []
            i = 0  # Reiniciar el índice
            while i < len(formula):
                if formula[i] == "!" and formula[i+1] in string.ascii_lowercase:
                    n = formula[i + 1]
                    vector1 = comprobador(diccionario_comprobador, n, None)
                    vector1 = isNegacion(vector1)
                    letra_aleatoria = generar_letra_aleatoria(formula_completa)
                    diccionario_comprobador[letra_aleatoria] = vector1
                    formula = borrador(formula, letra_aleatoria, n, " ")
                    break  # Salir del bucle interior después de manejar la negación
                i += 1  # Incrementar el índice

        # Manejar las disyunciones y conjunciones
        while "|" in formula or "&" in formula:
            vector1 = []
            vector2 = []
            i = 0  # Reiniciar el índice
            while i < len(formula):
                if formula[i] == "|" or formula[i] == "&":
                    n = formula[i + 1]
                    n_1 = formula[i - 1]

                    vector1 = comprobador(diccionario_comprobador, n, None)
                    vector2 = comprobador(diccionario_comprobador, None, n_1)

                    if formula[i] == "|":
                        vectores_1_2 = isDisyuncion(vector1, vector2)
                    if formula[i] == "&":
                        vectores_1_2 = isConjuncion(vector1, vector2)

                    letra_aleatoria = generar_letra_aleatoria(formula_completa)
                    diccionario_comprobador[letra_aleatoria] = vectores_1_2
                    formula = borrador(formula, letra_aleatoria, n, n_1)
                    break  # Salir del bucle interior después de manejar la disyunción o conjunción
                i += 1  # Incrementar el índice

        # Manejar las implicaciones y dobles implicaciones
        while ">" in formula or "=" in formula:
            vector1 = []
            vector2 = []
            i = 0  # Reiniciar el índice
            while i < len(formula):
                if formula[i] == ">" or formula[i] == "=":
                    n = formula[i + 1]
                    n_1 = formula[i - 1]

                    vector1 = comprobador(diccionario_comprobador, n, None)
                    vector2 = comprobador(diccionario_comprobador, None, n_1)

                    if formula[i] == ">":
                        vectores_1_2 = isImplicacion(vector1, vector2)
                    if formula[i] == "=":
                        vectores_1_2 = isDobleimplicacion(vector1, vector2)

                    letra_aleatoria = generar_letra_aleatoria(formula_completa)
                    diccionario_comprobador[letra_aleatoria] = vectores_1_2
                    formula = borrador(formula, letra_aleatoria, n, n_1)
                    break  # Salir del bucle interior después de manejar la implicación o doble implicación
                i += 1  # Incrementar el índice

        # Después de manejar cada operador, verificamos si quedan operadores por manejar
        if "!" not in formula and "|" not in formula and "&" not in formula and ">" not in formula and "=" not in formula:
            break  # Si no quedan operadores, detenemos el bucle


    return formula  # Asegurarse de devolver la fórmula final al final de la función

Funciones Logicas

In [8]:
def isNegacion(lista):
    """Opera las negaciones de la formula dada"""
    resultado = []  #Para mantener la letra identificadora
    for valor in lista:
        if valor == 0:
            resultado.append(1)
        elif valor == 1:
            resultado.append(0)

    return resultado


def isDisyuncion(lista1, lista2):
    """Observa dos listas y genera el resultado de disyuncion"""
    disyuncion = []
    for valor1, valor2 in zip(lista1, lista2):
        if valor1 == 1 and valor2 == 1:
            disyuncion.append(1)
        elif valor1 != valor2:
            disyuncion.append(1)
        elif valor1 == 0 and valor2 == 0:
            disyuncion.append(0)
    return disyuncion

def isConjuncion(lista1, lista2):
    """Observa dos listas y genera el resultado de conjuncion"""
    conjuncion = []
    for valor1, valor2 in zip(lista1, lista2):
        if valor1 == 1 and valor2 == 1:
            conjuncion.append(1)
        elif valor1 != valor2:
            conjuncion.append(0)
        elif valor1 == 0 and valor2 == 0:
            conjuncion.append(0)
    return conjuncion

def isImplicacion(lista1, lista2):
    """Observa dos listas y genera el resultado de implicacion"""
    implicacion = []
    for valor1, valor2 in zip(lista1, lista2):
        if valor1 == valor2:
            implicacion.append(1)
        elif valor1 == 1 and valor2 == 0:
            implicacion.append(0)
        elif valor1 == 0 and valor2 == 1:
            implicacion.append(1)
    return implicacion

def isDobleimplicacion(lista1, lista2):
    """Observa dos listas y genera el resultado de doble implicacion"""
    dobleimplicacion = []
    for valor1, valor2 in zip(lista1, lista2):
        if valor1 == valor2:
            dobleimplicacion.append(1)
        elif valor1 != valor2:
            dobleimplicacion.append(0)
    return dobleimplicacion

Generar tabla de verdad

In [39]:
def imprimir_tabla_verdad(formula, diccionario, letra_resultado):
    n_letras = contador_letras(formula)
    longitud = len(n_letras)
    if longitud == 0:
        print("Formula vacia")
        return
    """Va a imprimir la tabla de la verdad recorriendo el diccionario de las letras y sus respectivos valores"""
    letras_vistas = []  #lista para almacenar las letras que ya hemos visto y evitar duplicados
    valores = []  #lista para almacenar los valores de las letras que printearemos en tabla verdad
    for letra in formula:
        if letra.isalpha() and letra in diccionario and letra not in letras_vistas:
            letras_vistas.append(letra)
            valores.append(diccionario[letra])

    valores_ultima_letra = diccionario[letra_resultado]

    #IMPRIMIR LA TABLA DE VERDAD
    print("\n\nTabla de verdad:\n")
    encabezado = "\t".join(letras_vistas + [formula])
    separador = "-" * (len(encabezado) + len(letras_vistas) * 6)
    print(encabezado)
    print(separador)
    for fila, valor in zip(zip(*valores), valores_ultima_letra):
        fila_str = "\t".join(str(v) for v in fila) + "\t" + str(valor)
        print(fila_str)

Ejecutar ejercicio

In [43]:
print("Ingrese una formula válida: ")
formula = input()
formula = formula.replace(" ", "")
diccionario_comprobador = generador_diccionario(formula)

letra_resultado = regularizacion(formula, diccionario_comprobador)
imprimir_tabla_verdad(formula, diccionario_comprobador, letra_resultado)


Ingrese una formula válida: 
()u&p
2
La fórmula es correcta y puede procesarse.


Tabla de verdad:

u	p	()u&p
---------------------
0	0	0
1	0	0
0	1	0
1	1	1
